In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 92kB 9.4MB/s 
     |████████████████████████████████| 460kB 42.5MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


### **TF-IDF를 이용한 문장 유사도 계산** 

특정 문서 내에서 단어 빈도가 높을 수록, 그리고 전체 문서들 중 그 단어를 포함한 문서가 적을 수록 TF-IDF값이 높아진다.

- TF : 특정 단어 t가 문서 d 내에서 얼마나 자주 등장하는 지 나타내는 지표
- IDF : 특정 단어가 등장한 문서의 수를 나타내는 지표인 DF에 반비례하는 수 

*|D|는 전체 문서의 수를 의미한다.*

```
tfidf(t, d, D) = tf(t, d) * idf(t, D)
```


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Komoran
import networkx
import re
import os
import math

In [3]:
class RawTextReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile("/n")

    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in ch:
                yield s


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


기사 파일 경로 지정



> ```articles/Origin-Data``` :  크롤링한 원본 데이터, (기사 제목, 본문, 언론사를 포함한다.)  
> ```articles/Preprocessed-Data``` :  전처리를 마친 데이터 (중복 공백, 특수문자, 불용어 제거)  
> ```articles/Pretty-Data``` :  원본 데이터에서 본문을 문장 단위로 구분한 데이터  
> ```artitlces/StopWordList```:  불용어 리스트



In [5]:
BASE_DIR = "/content/gdrive/My Drive/Colab Notebooks/Text-preprocessing-Data/articles"
PREPROCESSED_PATH = os.path.join(BASE_DIR,"Preprocessed-Data")
PRETTY_PATH = os.path.join(BASE_DIR,"Pretty-Data")
ORIGIN_PATH = os.path.join(BASE_DIR,"Origin-Data")
SWORDS_PATH = os.path.join(BASE_DIR, "StopWordList.txt")

In [6]:
media_list = os.listdir(PREPROCESSED_PATH)

origin_article_list = os.listdir(os.path.join(PRETTY_PATH, media_list[0]))
proc_article_list = os.listdir(os.path.join(PREPROCESSED_PATH, media_list[0]))

origin_article_path = os.path.join(os.path.join(PRETTY_PATH, media_list[0]), origin_article_list[0])
proc_article_path = os.path.join(os.path.join(PREPROCESSED_PATH, media_list[0]), proc_article_list[0])

In [11]:
sentenceIter = RawTextReader(proc_article_path)
tokenizer = lambda sent: filter(lambda x: x[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent))

if not tokenizer: rgxSplitter = re.compile('[\\s.,:;-?!()"\']+')
sentSet = []

In [12]:
sents = set(filter(None, sentenceIter))
sents

{'경찰청 단행 인사 김경태 충북 경찰청 여성 보호 계장 송해 영 충북 청 홍보 계장 포함 경정 총경 승진 내정',
 '괴산 출신 김경태 계장 청주 울 고와 청주대학교 졸업 뒤 경찰 입문',
 '이후 청주 흥덕 경찰서 생활 안전 장과 경비 교통 장 충청 보안 사대장 생활 질서 계장 지내',
 '조준 영 기자 충북 경찰 연속 총경 승진자 나오',
 '청주 출신 청주 고와 경찰대 졸업 송해 영 계장 경위 경찰 입문 충북 청 기획 예산 계장 경무 계장 홍보 계장 역임'}

In [13]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(sents)
idf = tfidf_vectorizer.idf_
print(dict(zip(tfidf_vectorizer.get_feature_names(), idf)))

{'경무': 2.09861228866811, '경비': 2.09861228866811, '경위': 2.09861228866811, '경정': 2.09861228866811, '경찰': 1.4054651081081644, '경찰대': 2.09861228866811, '경찰서': 2.09861228866811, '경찰청': 2.09861228866811, '계장': 1.1823215567939547, '고와': 1.6931471805599454, '괴산': 2.09861228866811, '교통': 2.09861228866811, '기자': 2.09861228866811, '기획': 2.09861228866811, '김경태': 1.6931471805599454, '나오': 2.09861228866811, '내정': 2.09861228866811, '단행': 2.09861228866811, '보안': 2.09861228866811, '보호': 2.09861228866811, '사대장': 2.09861228866811, '생활': 2.09861228866811, '송해': 1.6931471805599454, '승진': 2.09861228866811, '승진자': 2.09861228866811, '안전': 2.09861228866811, '여성': 2.09861228866811, '역임': 2.09861228866811, '연속': 2.09861228866811, '예산': 2.09861228866811, '이후': 2.09861228866811, '인사': 2.09861228866811, '입문': 1.6931471805599454, '장과': 2.09861228866811, '조준': 2.09861228866811, '졸업': 1.6931471805599454, '지내': 2.09861228866811, '질서': 2.09861228866811, '청주': 1.4054651081081644, '청주대학교': 2.09861228866811, '총경': 1.693147

In [15]:
test_list= os.listdir(os.path.join(ORIGIN_PATH, os.listdir(ORIGIN_PATH)[0]))
test_path = os.path.join(os.path.join(ORIGIN_PATH, os.listdir(ORIGIN_PATH)[0]), test_list[0])

with open(test_path, 'r', encoding='utf-8') as f:
    title = f.readline()[:-1]
    content = f.readline()[:-1]

#### **Gensim 패키지를 이용한 TF_IDF 추출**

In [16]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
test_list= os.listdir(os.path.join(ORIGIN_PATH, os.listdir(ORIGIN_PATH)[0]))
test_path = os.path.join(os.path.join(ORIGIN_PATH, os.listdir(ORIGIN_PATH)[0]), test_list[0])

## 테스트할 원본 데이터
with open(test_path, 'r', encoding='utf-8') as f:
    title = f.readline()[:-1]
    content = f.readline()[:-1]
    print(title, content)

충북 경찰 4년 연속 총경 2명 배출 (청주=뉴스1) 조준영 기자 = 충북 경찰에서 4년 연속 2명의 총경 승진자가 나왔다. 경찰청은 7일 단행한 인사에서 김경태 충북경찰청 여성보호계장(50·일반 공채)과 송해영 충북청 홍보계장(49·경찰대 11기)을 포함한 경정 107명을 총경으로 승진 내정했다. 괴산 출신인 김경태 계장은 청주 운호고와 청주대학교를 졸업한 뒤 1993년 경찰에 입문했다. 이후 청주 흥덕경찰서 생활안전과장과 경비교통과장, 충청 보안수사대장, 생활질서계장 등을 지냈다. 청주 출신으로 청주고와 경찰대를 졸업한 송해영 계장은 1995년 경위로 경찰에 입문해 충북청 기획예산계장, 경무계장, 홍보계장 등을 역임했다.reason@news1.kr© 뉴스1코리아(), 무단 전재 및 재배포 금지


In [18]:
import gensim
import pprint
from gensim import corpora
from gensim import models
from nltk.tokenize import sent_tokenize
from gensim.utils import simple_preprocess
import numpy as np

doc_list = sent_tokenize(content)

doc_tokenized = [simple_preprocess(doc) for doc in doc_list]
dictionary = corpora.Dictionary()

# token 빈도
BoW_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in doc_tokenized]

for doc in BoW_corpus:
   print([[dictionary[id], freq] for id, freq in doc])
tfidf = models.TfidfModel(BoW_corpus, smartirs='ntc')

print('\nTF-IDF')
for doc in tfidf[BoW_corpus]:
   print([[dictionary[id], np.around(freq,decimals=2)] for id, freq in doc])

[['경찰에서', 1], ['기자', 1], ['나왔다', 1], ['뉴스', 1], ['명의', 1], ['승진자가', 1], ['연속', 1], ['조준영', 1], ['청주', 1], ['총경', 1], ['충북', 1]]
[['경정', 1], ['경찰대', 1], ['경찰청은', 1], ['공채', 1], ['김경태', 1], ['내정했다', 1], ['단행한', 1], ['명을', 1], ['송해영', 1], ['승진', 1], ['여성보호계장', 1], ['인사에서', 1], ['일반', 1], ['총경으로', 1], ['충북경찰청', 1], ['충북청', 1], ['포함한', 1], ['홍보계장', 1]]
[['청주', 1], ['김경태', 1], ['경찰에', 1], ['계장은', 1], ['괴산', 1], ['운호고와', 1], ['입문했다', 1], ['졸업한', 1], ['청주대학교를', 1], ['출신인', 1]]
[['청주', 1], ['경비교통과장', 1], ['등을', 1], ['보안수사대장', 1], ['생활안전과장과', 1], ['생활질서계장', 1], ['이후', 1], ['지냈다', 1], ['충청', 1], ['흥덕경찰서', 1]]
[['뉴스', 1], ['청주', 1], ['송해영', 1], ['충북청', 1], ['홍보계장', 1], ['경찰에', 1], ['계장은', 1], ['졸업한', 1], ['등을', 1], ['kr', 1], ['news', 1], ['reason', 1], ['경무계장', 1], ['경위로', 1], ['경찰대를', 1], ['금지', 1], ['기획예산계장', 1], ['무단', 1], ['역임했다', 1], ['입문해', 1], ['재배포', 1], ['전재', 1], ['청주고와', 1], ['출신으로', 1], ['코리아', 1]]

TF-IDF
[['경찰에서', 0.33], ['기자', 0.33], ['나왔다', 0.33], ['뉴스', 0.19], ['명의', 0.33], ['승진자

#### **sklearn의 TfidfVectorizer를 이용한 TF_IDF 추출**


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

sentenceIter = RawTextReader(proc_article_path)
docs = list(sentenceIter)

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(docs)
tfidf_vectorizer.vocabulary_


{'경무': 0,
 '경비': 1,
 '경위': 2,
 '경정': 3,
 '경찰': 4,
 '경찰대': 5,
 '경찰서': 6,
 '경찰청': 7,
 '계장': 8,
 '고와': 9,
 '괴산': 10,
 '교통': 11,
 '기자': 12,
 '기획': 13,
 '김경태': 14,
 '나오': 15,
 '내정': 16,
 '단행': 17,
 '보안': 18,
 '보호': 19,
 '사대장': 20,
 '생활': 21,
 '송해': 22,
 '승진': 23,
 '승진자': 24,
 '안전': 25,
 '여성': 26,
 '역임': 27,
 '연속': 28,
 '예산': 29,
 '이후': 30,
 '인사': 31,
 '입문': 32,
 '장과': 33,
 '조준': 34,
 '졸업': 35,
 '지내': 36,
 '질서': 37,
 '청주': 38,
 '청주대학교': 39,
 '총경': 40,
 '출신': 41,
 '충북': 42,
 '충청': 43,
 '포함': 44,
 '홍보': 45,
 '흥덕': 46}

In [20]:
inv_vocab = {v: k for k, v in tfidf_vectorizer.vocabulary_.items()}
inv_vocab

{0: '경무',
 1: '경비',
 2: '경위',
 3: '경정',
 4: '경찰',
 5: '경찰대',
 6: '경찰서',
 7: '경찰청',
 8: '계장',
 9: '고와',
 10: '괴산',
 11: '교통',
 12: '기자',
 13: '기획',
 14: '김경태',
 15: '나오',
 16: '내정',
 17: '단행',
 18: '보안',
 19: '보호',
 20: '사대장',
 21: '생활',
 22: '송해',
 23: '승진',
 24: '승진자',
 25: '안전',
 26: '여성',
 27: '역임',
 28: '연속',
 29: '예산',
 30: '이후',
 31: '인사',
 32: '입문',
 33: '장과',
 34: '조준',
 35: '졸업',
 36: '지내',
 37: '질서',
 38: '청주',
 39: '청주대학교',
 40: '총경',
 41: '출신',
 42: '충북',
 43: '충청',
 44: '포함',
 45: '홍보',
 46: '흥덕'}

In [21]:
dictionary = tfidf_vectorizer.get_feature_names()

for vec in tfidf_matrix.toarray():
    vector = [[inv_vocab[idx], np.around(tfidf,decimals=2)] for idx, tfidf in enumerate(vec) if tfidf > 0]
    print(vector)


[['경찰', 0.26], ['기자', 0.39], ['나오', 0.39], ['승진자', 0.39], ['연속', 0.39], ['조준', 0.39], ['총경', 0.32], ['충북', 0.26]]
[['경정', 0.24], ['경찰청', 0.48], ['계장', 0.27], ['김경태', 0.19], ['내정', 0.24], ['단행', 0.24], ['보호', 0.24], ['송해', 0.19], ['승진', 0.24], ['여성', 0.24], ['인사', 0.24], ['총경', 0.19], ['충북', 0.32], ['포함', 0.24], ['홍보', 0.19]]
[['경찰', 0.26], ['계장', 0.22], ['고와', 0.32], ['괴산', 0.39], ['김경태', 0.32], ['입문', 0.32], ['졸업', 0.32], ['청주', 0.26], ['청주대학교', 0.39], ['출신', 0.32]]
[['경비', 0.24], ['경찰서', 0.24], ['계장', 0.14], ['교통', 0.24], ['보안', 0.24], ['사대장', 0.24], ['생활', 0.49], ['안전', 0.24], ['이후', 0.24], ['장과', 0.24], ['지내', 0.24], ['질서', 0.24], ['청주', 0.16], ['충청', 0.24], ['흥덕', 0.24]]
[['경무', 0.24], ['경위', 0.24], ['경찰', 0.16], ['경찰대', 0.24], ['계장', 0.54], ['고와', 0.19], ['기획', 0.24], ['송해', 0.19], ['역임', 0.24], ['예산', 0.24], ['입문', 0.19], ['졸업', 0.19], ['청주', 0.32], ['출신', 0.19], ['충북', 0.16], ['홍보', 0.19]]


#### **sklearn의 TfidfTransformer를 이용한 TF_IDF 추출**


In [22]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer 
import pandas as pd

sentenceIter = RawTextReader(proc_article_path)
docs = list(sentenceIter)

cv=CountVectorizer() 
word_count_vector=cv.fit_transform(docs)

tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"]) 
 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])

,idf_weights
계장,1.182322
충북,1.405465
경찰,1.405465
청주,1.405465
김경태,1.693147
졸업,1.693147
고와,1.693147
입문,1.693147
총경,1.693147
출신,1.693147


In [23]:
# count matrix 
count_vector=cv.transform(docs) 
 
# tf-idf scores 
tf_idf_vector=tfidf_transformer.transform(count_vector)

feature_names = cv.get_feature_names() 
 
#get tfidf vector for first document 
first_document_vector=tf_idf_vector[0] 

#print the scores 
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
기자,0.390794
나오,0.390794
연속,0.390794
조준,0.390794
승진자,0.390794
총경,0.315290
경찰,0.261719
충북,0.261719
포함,0.000000
역임,0.000000
